<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="250" align="center">

**PROCESAMIENTO DE LENGUAJE NATURAL**

*DESAFÍO Nº3 - CUSTOM EMBEDDINGS CON GENSIM - JUAN I. MUNAR*



***Objetivo.*** *Utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto.*

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

**1. DATOS**

Utilizaremos como dataset los dos poemas épicos de José Hernández: "El Gaucho" y "La vuelta de Martín Fierro".Entrenaremos así embeddings gauchos. Se obtuvieron los textos de: https://www.textos.info/jose-hernandez

In [ ]:
# Armo el dataset gaucho con los dos libros de José Hernández
df = pd.read_csv('MARTIN_FIERRO.txt', sep='/n', header=None)
df.head(7)

<ipython-input-2-c9266a5a5b2a>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('MARTIN_FIERRO.txt', sep='/n', header=None)


,0
0,I
1,Aquí me pongo a cantar
2,"al compás de la vigüela,"
3,que el hombre que lo desvela
4,"una pena estrordinaria,"
5,como la ave solitaria
6,con el cantar se consuela.


In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 7353


**2. PREPROCESAMIENTO**

In [ ]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[1:7]

[['aquí', 'me', 'pongo', 'a', 'cantar'],
 ['al', 'compás', 'de', 'la', 'vigüela'],
 ['que', 'el', 'hombre', 'que', 'lo', 'desvela'],
 ['una', 'pena', 'estrordinaria'],
 ['como', 'la', 'ave', 'solitaria'],
 ['con', 'el', 'cantar', 'se', 'consuela']]

**3. CREACIÓN DE VECTORES**

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=2,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=4,       # cant de palabras antes y desp de la predicha
                     vector_size=20,       # dimensionalidad de los vectores
                     negative=5,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 7353


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 2552


**4. ENTRENAMIENTO DE EMBEDDINGS**

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 172061.453125
Loss after epoch 1: 150942.359375
Loss after epoch 2: 134880.3125
Loss after epoch 3: 125614.5
Loss after epoch 4: 121709.125
Loss after epoch 5: 119111.75
Loss after epoch 6: 117530.125
Loss after epoch 7: 113111.75
Loss after epoch 8: 112430.75
Loss after epoch 9: 110610.5
Loss after epoch 10: 108583.5
Loss after epoch 11: 106679.125
Loss after epoch 12: 105809.75
Loss after epoch 13: 105604.625
Loss after epoch 14: 103904.125
Loss after epoch 15: 103108.75
Loss after epoch 16: 101724.75
Loss after epoch 17: 100896.5
Loss after epoch 18: 102630.5
Loss after epoch 19: 98960.75
Loss after epoch 20: 98250.5
Loss after epoch 21: 95206.5
Loss after epoch 22: 94641.5
Loss after epoch 23: 93680.25
Loss after epoch 24: 91919.75
Loss after epoch 25: 91415.5
Loss after epoch 26: 90973.5
Loss after epoch 27: 88367.5
Loss after epoch 28: 88386.75
Loss after epoch 29: 87881.75
Loss after epoch 30: 86848.75
Loss after epoch 31: 85158.5
Loss after epoch 32: 85564.2

(2087870, 3484000)

**5. ENSAYO**

Buscar palabras en:
https://www.folkloretradiciones.com.ar/literatura/Diccionario%20Gaucho.pdf

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["gaucho"], topn=5)

[('sogazo', 0.8504615426063538),
 ('perseguido', 0.7970082759857178),
 ('tratándome', 0.7969235181808472),
 ('vago', 0.7813240885734558),
 ('cormillo', 0.7721196413040161)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["yapa"], topn=5)

[('madre', 0.061860159039497375),
 ('presentes', 0.055455632507801056),
 ('filas', 0.020827623084187508),
 ('visto', 0.01107831671833992),
 ('hay', -0.014213834889233112)]

In [ ]:
# Palabras que MÁS se relacionan con Dios:
w2v_model.wv.most_similar(positive=["criador"], topn=10)

[('rogué', 0.8824184536933899),
 ('dolorida', 0.8390344977378845),
 ('alma', 0.8264591693878174),
 ('trabajando', 0.8098248839378357),
 ('reyuno', 0.8092127442359924),
 ('pobreza', 0.8053876161575317),
 ('tenganló', 0.8043805360794067),
 ('vale', 0.7936849594116211),
 ('malos', 0.7872698307037354),
 ('llena', 0.7856756448745728)]

In [ ]:
# Palabras que MÁS se relacionan con la ciencia, cencia en gaucho:
w2v_model.wv.most_similar(positive=["cencia"], topn=10)

[('escasa', 0.9061871767044067),
 ('chacota', 0.8753162026405334),
 ('presumir', 0.8502817153930664),
 ('guardar', 0.845954179763794),
 ('gala', 0.8217167854309082),
 ('dudo', 0.8206869959831238),
 ('dicha', 0.8197711110115051),
 ('ablande', 0.8104197382926941),
 ('pensamiento', 0.8032798767089844),
 ('blanda', 0.8007001876831055)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["mujer"], topn=5)

[('pensar', 0.8490882515907288),
 ('madre', 0.8271868228912354),
 ('parida', 0.7843178510665894),
 ('inocencia', 0.7840858697891235),
 ('cueva', 0.7757599353790283)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["hombre"], topn=5)

[('astucia', 0.8268446326255798),
 ('altanero', 0.7728117108345032),
 ('somete', 0.7637494206428528),
 ('ser', 0.7507352232933044),
 ('agarraba', 0.7437267303466797)]

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_padre = w2v_model.wv.get_vector("padre")
print(vector_padre)

[-0.4326783   0.3492117  -0.20802353 -0.75731754 -0.2663071  -0.83053535
  0.8683516   0.9627751  -0.33420897  0.17576188  0.6461766  -0.6209972
  0.08426825  0.69366264 -0.01595506  0.2165462   0.7143331   0.7575728
 -0.07563095 -1.9703963 ]


In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_padre)

[('padre', 1.0),
 ('otorgó', 0.8616623878479004),
 ('marido', 0.8150213360786438),
 ('eterno', 0.7940366864204407),
 ('leche', 0.7914135456085205),
 ('señor', 0.7788155674934387),
 ('soldao', 0.7781060934066772),
 ('abandonado', 0.7516515851020813),
 ('maldecía', 0.7495874166488647),
 ('encargao', 0.7452136874198914)]

In [ ]:
# Test de analogía
vector_padre = w2v_model.wv.get_vector("padre")
vector_madre = w2v_model.wv.get_vector("madre")
vector_mujer = w2v_model.wv.get_vector("mujer")

# padre - hombre/esposo/marido = madre - mujer
nuevo_vector = vector_padre + vector_mujer - vector_madre

# Busquemos los valores más similares al nuevo vector
w2v_model.wv.most_similar(nuevo_vector)

[('padre', 0.8552191853523254),
 ('plan', 0.8152356743812561),
 ('encargao', 0.8053070902824402),
 ('guarida', 0.7983205914497375),
 ('flete', 0.7877184748649597),
 ('marido', 0.7774718403816223),
 ('cuartel', 0.7768259644508362),
 ('mataron', 0.7699558138847351),
 ('frente', 0.7626943588256836),
 ('hocico', 0.7604458928108215)]

El resultado de este test no es bueno. Aparece la palabra marido recién en la quinta posición (sexta si se cuenta padre). Esto parece ser una característica propia del texto más que de los embeddings. Las formas más sencillas de testear involucran el género, geografía, rangos políticos/militares, etc. Ninguno de estos aspectos es fácil de detallar en el texto, tal vez lo mejor que haya es la combinación de embeddings anterior.

Las relaciones de género y familia cuando fue escrito el texto, y en el contexto en el que fue escrito, son muy diferentes a las actuales. La mujer está asociada claramente a roles maternos/caseros. Entre palabras similares a mujer aparecen: pensar, madre, parida, cueva, inocencia, etc. El hombre en este poema épico suele ser errante o vagabundo, con atributos que lo alejan de la relación *padre-marido / madre-mujer*. Así vemos como palabras más similares a hombre a: astucia, altanero, somete, etc. Este aspecto se suma a la gran pluma de Hernández, que suele variar mucho el lenguaje al describir las conductas, lo que dificulta entrenar embeddings por la menor frecuencia de aparición de palabras.

**6. VISUALIZACIÓN**

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

**7. CONCLUSIONES**

Los embeddings entrenados funcionan muy bien para palabras gaúchas, cuando uno se aparta de ellas, el funcionamiento es algo más inestable pero el resultado obtenido es aceptable. El propio entrenamiento de los embeddings resulta inestable, en parte por la baja longitud del texto, en parte por la riqueza de la pluma de Hernández. Los problemas provienen de la baja frecuencia de aparición de palabras. Dependiendo de la frecuencia mínima y del tamaño de la ventana considerado para el entrenamiento, cambian drásticamente los embeddings.